# Download data from ERDDAP using GRIDDAP
### Written by Minh Phan

**NOTE: YOU CAN SKIP THIS NOTEBOOK IF YOU ONLY WANT TO DOWNLOAD SAMPLE DATA OF OUR DATASET SINCE WE DO NOT USE ANY DATA FROM ERDDAP IN OUR FINAL PRODUCT.**

This tutorial serves to provide one of many ways a user can download data from NOAA's ERDDAP HTML link template. Unlike NASA's EarthData, you don't need to register an account to download data from NOAA ERRDAP.

## Import necessary libraries

In [2]:
! pip install erddap-python

In [1]:
import xarray as xr
from erddapClient import ERDDAP_Griddap

## Download data

Unlike NASA Earthdata, we can slice dataset spatially directly for every server request. **We do not recommend slicing down and sending multiple requests, as it may put you into the organization's blacklist, hence the discontinuation of downloading using URLs approach. Instead, utilize [the library](https://pypi.org/project/erddap-python/) wrote by ERDDAP team to help slicing and downloading data.**

**DISCLAIMER**: make sure that the dataset is hosted on the correct database. For example, the datasetID 'hawaiii_soest_66d3_10d8_0f3c' is hosted on the University of Hawaii's ERDDAP database and is not available to download on NOAA database. Make sure to add the correct database URL to the dictionary for the downloader to work

In [3]:
# erddap website (server) & datasetID
remote = ERDDAP_Griddap('https://pae-paha.pacioos.hawaii.edu/erddap', 'srtm30plus_v11_bathy')

In [ ]:
# get data
xSubset = ( remote.setResultVariables('elev')
                 .setSubset(latitude=slice(-12, 32),
                            longitude=slice(42, 102))
                 .getxArray() )

## Get to know GRIDDAP URL link template [DEPRECATED]

One of the datasets that we used in this project is [Reanalysis Data ERA5 monthly 3d Wind velocities](https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_66d3_10d8_0f3c.html). For the default options, select file type as .nc (NetCDF3-binary file) and click 'Just generate the URL,' you can see the URL is then https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_66d3_10d8_0f3c.nc?u[(2023-04-01T00:00:00Z):1:(2023-04-01T00:00:00Z)][(10.0):1:(1000.0)][(-90.0):1:(90.0)][(0.0):1:(359.75)],v[(2023-04-01T00:00:00Z):1:(2023-04-01T00:00:00Z)][(10.0):1:(1000.0)][(-90.0):1:(90.0)][(0.0):1:(359.75)] and it follows this template:

<div style="display: inline">https://apdrc.soest.hawaii.edu/erddap/griddap</div>/<b>DATASET_ID</b>.<b>FILE_EXTENSION</b> 

following by

<b>VARIABLE_NAME1</b> [(<b>START_DIM1</b>):<b>STRIDE_DIM1</b>:(<b>STOP_DIM1</b>)][(<b>START_DIM2</b>):<b>STRIDE_DIM2</b>:(<b>STOP_DIM2</b>)][(<b>START_DIM3</b>):<b>STRIDE_DIM3</b>:(<b>STOP_DIM3</b>)]... 

and then


<b>VARIABLE_NAME2</b>[(<b>START_DIM1</b>):<b>STRIDE_DIM1</b>:(<b>STOP_DIM1</b>)][(<b>START_DIM2</b>):<b>STRIDE_DIM2</b>:(<b>STOP_DIM2</b>)][(<b>START_DIM3</b>):<b>STRIDE_DIM3</b>:(<b>STOP_DIM3</b>)]... 

and so on for all the variables. To illustrate this template, you can liken it to downloading every variable as an Xarray DataArray, and combine them together based on the coordinates. In the URL above, the order and specifications of the dimension slicing options must be the consistent for all variables, just like how Xarray combining by coords only works if the coordinates are so. 

The template is also well-explained in [GRIDDAP documentation](https://coastwatch.pfeg.noaa.gov/erddap/griddap/documentation.html).

We can all agree that trying to write down every template for different datasets is error-prone and tedious, so you are more than welcome to copy and modify your template using the original file generator Python formatted strings to help with generating URLs for downloads. Since URLs vary greatly form dataset to dataset, there is no ideal URL template to consider, but we will still develop a function to generalize the URL generator to the best of my knowledge.

**NOTE**: for the URL template, we focus on downloading smaller chunks of the dataset, sliced temporally just like downloading from the EarthData database so that the remote server would not be broken handling the request. We also recommend you to download variables together in one NetCDF3 file (multiple variable names in one URL) instead of individual variables as it takes time piecing them all together.